[View in Colaboratory](https://colab.research.google.com/github/iamthejao/DeepLearningLab-A1/blob/master/Assignment.ipynb)

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [0]:
gpu_available = tf.test.gpu_device_name()

In [0]:
def create_dataset(w_star, x_range, sample_size, sigma, seed=None):
  random_state = np.random.RandomState(seed)
  x = random_state.uniform(x_range[0] , x_range[1] , (sample_size))
  X = np.zeros((sample_size , w_star.shape[0]))
  for i in range(sample_size):
    X[i, 0] = 1.
    for j in range(1, w_star.shape[0]):
      X[i, j] = x[i]**j
  y = X.dot(w_star).flatten()
  if sigma > 0:
    y += random_state.normal(0.0 , sigma , sample_size )
  return X, y



In [0]:
w_star = np.array([-8, -4, 2, 1])[np.newaxis].T
x_range = [-3,2]
sample_size = 100
seed_train = 0
seed_validation = 1
sigma = 0.5

In [0]:
random_state = np.random.RandomState(seed)
x = random_state.uniform(x_range[0] , x_range[1] , (sample_size))
X = np.zeros((sample_size , w_star.shape[0]))
for i in range(sample_size):
  X[i, 0] = 1.
  for j in range(1, w_star.shape[0]):
    X[i, j] = x[i]**j
y = X.dot(w_star).flatten()

In [0]:
X_train, y_train = create_dataset(w_star, x_range, sample_size, sigma, seed=seed_train)
X_val, y_val = create_dataset(w_star, x_range, sample_size, sigma, seed=seed_validation)

In [0]:
# Defining the Tensorflow environment to calculate the problem
def gradient_descent_linearRegression(Xtrain, ytrain, learning_rate, iterations, session=None, validation=None):
  
  # Setting TF variables
  X = tf.placeholder(tf.float32, shape=(None, X.shape[-1]))
  y = tf.placeholder(tf.float32, shape=(None,))
  w = tf.Variable(tf.zeros((X.shape[-1],1)), trainable=True)
  
  # Mounting loss function
  predicion = tf.matmul(X, w).flatten()
  loss = tf.reduce_mean(tf.square(y - prediction))
  
  # Selecting optimizer
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  train = optimizer.minimize(loss)
  
  # Creating session
  if session is None:
    session = tf.Session()
    
  initializer = tf.global_variables_initializer()
  session.run(initializer)
  
  # Iterating training
  for i in range(iterations):
    l, _ = session.run([loss, train], feed_dict={X:Xtrain, y: ytrain})
    print("Iteration {0}. Loss: {1}.".format(i, l))
  
  # Check if validation set is given
  if validation is not None:
    Xval, yval = validation
    l = session.run(loss, feed_dict={X:Xval, y:yval})
    
  
  
   
  
  